# **Import Modules**

In [ ]:
!pip install nlpaug transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
import nlpaug.augmenter.word as naw

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Embedding, LSTM, Dense, Dropout,
                                     GlobalMaxPooling1D, Conv1D, GRU,
                                     Bidirectional, BatchNormalization,
                                     SpatialDropout1D, MaxPooling1D,
                                     Activation, GlobalAveragePooling1D)
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight

warnings.filterwarnings('ignore')

# **Memuat Data**

In [ ]:
!wget --no-check-certificate https://raw.githubusercontent.com/rioooranteai/nlp-project/main/Analisis%20Sentimen%20-%20Coretax/Dataset/Prep_Version-2/With-Lemma/Dataset_roberta.csv -O "/content/Dataset.csv"

--2025-04-29 04:16:16--  https://raw.githubusercontent.com/rioooranteai/nlp-project/main/Analisis%20Sentimen%20-%20Coretax/Dataset/Prep_Version-2/With-Lemma/Dataset_roberta.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3292644 (3.1M) [text/plain]
Saving to: ‘/content/Dataset.csv’

/content/Dataset.cs 100%[===================>]   3.14M  --.-KB/s    in 0.06s   

2025-04-29 04:16:17 (57.1 MB/s) - ‘/content/Dataset.csv’ saved [3292644/3292644]



In [ ]:
df = pd.read_csv("/content/Dataset.csv", usecols=['full_text', 'sentiment'])
df.head()

,full_text,sentiment
0,hai kak. mohon maaf atas tidaknyama nya . apab...,neutral
1,hai kak. mohon maaf atas tidaknyama nya . apa ...,neutral
2,hai kak . apa kah alamat kakak pindah ke wilay...,neutral
3,hhaaai! sha open jasa ya ges *lapor spt masa p...,neutral
4,pajak yang bayar lewat sakti seharusnya 3.480....,neutral


In [ ]:
df.tail()

,full_text,sentiment
14690,gw curiga ini server sakti dipake buat coretax...,negative
14691,kak hari ( kringpajak ) coba buka dulu coretax...,neutral
14692,kayak pernah denger tentang coretax deh. . . (...,negative
14693,setuju sangat setuju! bagai individu yang baya...,negative
14694,sering dapet kinini data kak seperti coretax,neutral


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14695 entries, 0 to 14694
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  14694 non-null  object
 1   sentiment  14695 non-null  object
dtypes: object(2)
memory usage: 229.7+ KB


In [ ]:
df['full_text'] = df['full_text'].str.strip()

# **Split Data**

In [ ]:
data = df.copy(deep=True)
data.dropna(inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14694 entries, 0 to 14694
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  14694 non-null  object
 1   sentiment  14694 non-null  object
dtypes: object(2)
memory usage: 344.4+ KB


In [ ]:
le = LabelEncoder()

data['sentiment'] = le.fit_transform(data['sentiment'])

In [ ]:
X = data['full_text']
y = data['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,random_state=42)

# **Handling Unbalance Data**

## **Nlpaug**

In [ ]:
X_train_pos = X_train[y_train == 2]
X_train_pos.reset_index(drop=True, inplace=True)

In [ ]:
aug_insert = naw.ContextualWordEmbsAug(
    model_path='indolem/indobertweet-base-uncased',
    model_type='bert',
    action='insert',
    device='cuda'
)

aug_subs = naw.ContextualWordEmbsAug(
    model_path='indolem/indobertweet-base-uncased',
    model_type='bert',
    action='substitute',
    device='cuda'
)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
augmented_data = []

for data_text in X_train_pos:
    for _ in range(10):
        augmented_text_insert = aug_insert.augment(data_text)
        augmented_data.append(augmented_text_insert)

        augmented_text_subs = aug_subs.augment(data_text)
        augmented_data.append(augmented_text_subs)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
X_train_balance = pd.concat([X_train, pd.Series(augmented_data)], ignore_index=True)
y_train_balance = pd.concat([y_train, pd.Series([2] * len(augmented_data))], ignore_index=True)

# **Tokenisasi**

In [ ]:
all_words = []

for text in data['full_text']:
    words = text.lower().split()
    all_words.extend(words)

word_counts = Counter(all_words)

sorted_counts = word_counts.most_common()

total_freq = sum(word_counts.values())

cumulative = 0
num_words = 0

for word, count in sorted_counts:
    cumulative += count
    num_words += 1
    if cumulative / total_freq >= 0.95:
        break

print(f"Jumlah kata yang mencakup 95% dari total frekuensi: {num_words}")

Jumlah kata yang mencakup 95% dari total frekuensi: 4176


In [ ]:
VOCAB_SIZE = 4176
MAX_LEN = 100
OOV_TOKEN = '<OOV>'
EMBEDDING_DIM = 100
BATCH_SIZE = 64

In [ ]:
tk = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tk.fit_on_texts(X_train_balance)

seq_train = tk.texts_to_sequences(X_train_balance)
pad_train = pad_sequences(seq_train, maxlen=MAX_LEN, padding='post', truncating='post')

seq_test = tk.texts_to_sequences(X_test)
pad_test = pad_sequences(seq_test, maxlen=MAX_LEN, padding='post', truncating='post')

In [ ]:
y_train_encode = tf.keras.utils.to_categorical(y_train_balance, num_classes=3)
y_test_encode = tf.keras.utils.to_categorical(y_test, num_classes=3)

# **Pemodelan dengan Deep Learning**

In [ ]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced',
                               classes=classes,
                               y=y_train)

class_weight = dict(zip(classes, weights))

In [ ]:
STEPS_PER_EPOCH = int(len(pad_train) / BATCH_SIZE)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=STEPS_PER_EPOCH*5,
    decay_rate=0.9,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

class Mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.92 and logs.get('val_accuracy')>0.92):
      print('\nAkurasi telah mencapai > 92%')
      self.model.stop_training = True

stop_train = Mycallback()

callbacks = [early_stopping, checkpoint, stop_train]

## **LSTM**

In [ ]:
model = tf.keras.Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                              input_length=MAX_LEN,
                              embeddings_regularizer=tf.keras.regularizers.l2(1e-5)),

    Bidirectional(LSTM(16, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(8)),
    Dropout(0.3),
    Dense(4, activation='relu'),
    Dense(3, activation='softmax')
])

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [ ]:
hist = model.fit(pad_train, y_train_encode,
                    batch_size=BATCH_SIZE,
                    class_weight=class_weight,
                    epochs=100,
                    validation_data=(pad_test, y_test_encode),
                    verbose=2,
                    callbacks=callbacks,
                    steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.10752, saving model to best_model.h5


384/384 - 14s - 36ms/step - accuracy: 0.5650 - loss: 0.8854 - precision_6: 0.7601 - recall_6: 0.5145 - val_accuracy: 0.1075 - val_loss: 1.1912 - val_precision_6: 0.2786 - val_recall_6: 0.0265
Epoch 2/100

Epoch 2: val_accuracy improved from 0.10752 to 0.10786, saving model to best_model.h5


384/384 - 1s - 1ms/step - accuracy: 0.4359 - loss: 0.4852 - precision_6: 0.8947 - recall_6: 0.4359 - val_accuracy: 0.1079 - val_loss: 1.1887 - val_precision_6: 0.2821 - val_recall_6: 0.0262
Epoch 3/100

Epoch 3: val_accuracy improved from 0.10786 to 0.59170, saving model to best_model.h5


384/384 - 9s - 23ms/step - accuracy: 0.6720 - loss: 0.4976 - precision_6: 0.9248 - recall_6: 0.5472 - val_accuracy: 0.5917 - val_loss: 0.8916 - val_precision_6: 0.8315 - val_recall_6: 0.2855
Epoch 4/100

Epoch 4: val_accuracy improved from 0.59170 to 0.59408, saving model to best_model.h5


384/384 - 1s - 2ms/step - accuracy: 0.7692 - loss: 0.5168 - precision_6: 0.9600 - recall_6: 0.6154 - val_accuracy: 0.5941 - val_loss: 0.8923 - val_precision_6: 0.8307 - val_recall_6: 0.2838
Epoch 5/100

Epoch 5: val_accuracy improved from 0.59408 to 0.75536, saving model to best_model.h5


...

384/384 - 10s - 26ms/step - accuracy: 0.9086 - loss: 0.2555 - precision_6: 0.9200 - recall_6: 0.8903 - val_accuracy: 0.7979 - val_loss: 0.5900 - val_precision_6: 0.8064 - val_recall_6: 0.7836
Epoch 10/100

Epoch 10: val_accuracy improved from 0.79789 to 0.80027, saving model to best_model.h5


384/384 - 1s - 1ms/step - accuracy: 0.8974 - loss: 0.3766 - precision_6: 0.9211 - recall_6: 0.8974 - val_accuracy: 0.8003 - val_loss: 0.5879 - val_precision_6: 0.8080 - val_recall_6: 0.7877
Epoch 11/100

Epoch 11: val_accuracy did not improve from 0.80027
384/384 - 10s - 26ms/step - accuracy: 0.9305 - loss: 0.2072 - precision_6: 0.9361 - recall_6: 0.9237 - val_accuracy: 0.7829 - val_loss: 0.6530 - val_precision_6: 0.7928 - val_recall_6: 0.7734
Epoch 12/100

Epoch 12: val_accuracy did not improve from 0.80027
384/384 - 1s - 2ms/step - accuracy: 0.9231 - loss: 0.1287 - precision_6: 0.9211 - recall_6: 0.8974 - val_accuracy: 0.7839 - val_loss: 0.6530 - val_precision_6: 0.7936 - val_recall_6: 0.7720
Epoch 13/100

Epoch 13: val_accuracy improved from 0.80027 to 0.80436, saving model to best_model.h5


384/384 - 8s - 20ms/step - accuracy: 0.9425 - loss: 0.1733 - precision_6: 0.9468 - recall_6: 0.9386 - val_accuracy: 0.8044 - val_loss: 0.6145 - val_precision_6: 0.8081 - val_recall_6: 0.7952
Epoch 14/100

Epoch 14: val_accuracy did not improve from 0.80436
384/384 - 0s - 1ms/step - accuracy: 0.9487 - loss: 0.1168 - precision_6: 0.9487 - recall_6: 0.9487 - val_accuracy: 0.8023 - val_loss: 0.6191 - val_precision_6: 0.8074 - val_recall_6: 0.7931
Epoch 15/100

Epoch 15: val_accuracy improved from 0.80436 to 0.81252, saving model to best_model.h5


384/384 - 10s - 25ms/step - accuracy: 0.9530 - loss: 0.1475 - precision_6: 0.9556 - recall_6: 0.9503 - val_accuracy: 0.8125 - val_loss: 0.6062 - val_precision_6: 0.8163 - val_recall_6: 0.8088
Epoch 16/100

Epoch 16: val_accuracy improved from 0.81252 to 0.81320, saving model to best_model.h5


384/384 - 1s - 2ms/step - accuracy: 0.8974 - loss: 0.2103 - precision_6: 0.9211 - recall_6: 0.8974 - val_accuracy: 0.8132 - val_loss: 0.6074 - val_precision_6: 0.8166 - val_recall_6: 0.8074
Epoch 17/100

Epoch 17: val_accuracy did not improve from 0.81320
384/384 - 7s - 19ms/step - accuracy: 0.9613 - loss: 0.1266 - precision_6: 0.9633 - recall_6: 0.9591 - val_accuracy: 0.8050 - val_loss: 0.6616 - val_precision_6: 0.8082 - val_recall_6: 0.8003
Epoch 18/100

Epoch 18: val_accuracy did not improve from 0.81320
384/384 - 0s - 1ms/step - accuracy: 0.9231 - loss: 0.1175 - precision_6: 0.9231 - recall_6: 0.9231 - val_accuracy: 0.8064 - val_loss: 0.6555 - val_precision_6: 0.8114 - val_recall_6: 0.8020
Epoch 19/100

Epoch 19: val_accuracy did not improve from 0.81320
384/384 - 8s - 20ms/step - accuracy: 0.9631 - loss: 0.1235 - precision_6: 0.9648 - recall_6: 0.9610 - val_accuracy: 0.8016 - val_loss: 0.7887 - val_precision_6: 0.8041 - val_recall_6: 0.7976
Epoch 20/100

Epoch 20: val_accuracy did

Evaluate Model

In [ ]:
y_test_probs = model.predict(pad_test)
y_pred = np.argmax(y_test_probs, axis=1)
y_true = np.argmax(y_test_encode, axis=1)

target_names = le.inverse_transform([0,1,2])

print(classification_report(y_true, y_pred, target_names=target_names))

92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
              precision    recall  f1-score   support

    negative       0.82      0.76      0.79      1348
     neutral       0.91      0.82      0.86      1430
    positive       0.26      0.65      0.37       161

    accuracy                           0.78      2939
   macro avg       0.66      0.74      0.67      2939
weighted avg       0.83      0.78      0.80      2939



## **CNN**

In [ ]:
cnn_model = Sequential([
    # 1. Embedding + SpatialDropout
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
              input_length=MAX_LEN,
              embeddings_regularizer=tf.keras.regularizers.l2(1e-5)),
    # Reshape the output of the Embedding layer to 3D
    tf.keras.layers.Reshape((MAX_LEN, EMBEDDING_DIM)),  # Add this line
    SpatialDropout1D(0.3),

    # 2. Conv Block 1
    Conv1D(64, 3, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # 3. Conv Block 2
    Conv1D(64, 5, padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    # 4. Global Pooling
    GlobalAveragePooling1D(),

    # 5. Fully Connected
    Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    Dropout(0.5),

    # 6. Output
    Dense(3, activation='softmax')
])

cnn_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_6             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
cnn_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

cnn_hist = cnn_model.fit(pad_train, y_train_encode,
                    epochs=100,
                    batch_size=BATCH_SIZE,
                    validation_data=(pad_test, y_test_encode),
                    verbose=2,
                    callbacks=callbacks,
                    steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/100


ValueError: Unknown variable: <Variable path=sequential_4/embedding_7/embeddings, shape=(6213, 100), dtype=float32, value=[[-0.00053514 -0.03063952  0.0261015  ... -0.01596267  0.02950436
   0.01930836]
 [ 0.01141304  0.02812502 -0.02740994 ... -0.0302422   0.03470713
   0.03468462]
 [-0.02912616  0.04708688  0.03848168 ... -0.02813857  0.01659379
  -0.03159082]
 ...
 [ 0.02955835  0.03898885  0.04103046 ...  0.02214685 -0.02832992
  -0.04777972]
 [ 0.00844231 -0.01098595  0.03203896 ...  0.012782    0.01895117
   0.04217687]
 [-0.04081379  0.04288638  0.01586945 ... -0.01320772  0.0298248
  -0.00310557]]>. This optimizer can only be called for the variables it was originally built with. When working with a new set of variables, you should recreate a new optimizer instance.

Evaluate Model

In [ ]:
y_test_probs = model.predict(pad_test)
y_pred = np.argmax(y_test_probs, axis=1)
y_true = np.argmax(y_test_encode, axis=1)

target_names = le.inverse_transform([0,1,2])

print(classification_report(y_true, y_pred, target_names=target_names))

## **GRU**

In [ ]:
gru_model = Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                                 input_length=MAX_LEN,
                                 embeddings_regularizer=tf.keras.regularizers.l2(1e-5)),

    SpatialDropout1D(0.3),

    Bidirectional(tf.keras.layers.GRU(64, recurrent_dropout=0.2,
                                      reset_after=True)),

    Dense(64, activation='relu',
                kernel_regularizer=tf.keras.regularizers.l2(1e-4)),

    BatchNormalization(),

    Dropout(0.4),

    Dense(3, activation='softmax')
])

In [ ]:
gru_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
gru_hist = gru_model.fit(pad_train, y_train_encode,
                    epochs=100,
                    batch_size=BATCH_SIZE,
                    validation_data=(pad_test, y_test_encode),
                    verbose=2,
                    callbacks=callbacks,
                    steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/100

Epoch 1: val_accuracy did not improve from 0.72497
182/182 - 97s - 532ms/step - accuracy: 0.8679 - loss: 0.3439 - val_accuracy: 0.5988 - val_loss: 0.8658
Epoch 2/100

Epoch 2: val_accuracy improved from 0.72497 to 0.82407, saving model to best_model.h5


182/182 - 90s - 495ms/step - accuracy: 0.9078 - loss: 0.2562 - val_accuracy: 0.8241 - val_loss: 0.4953
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.82407
182/182 - 14s - 79ms/step - accuracy: 0.9209 - loss: 0.2357 - val_accuracy: 0.8158 - val_loss: 0.5054
Epoch 4/100

Epoch 4: val_accuracy improved from 0.82407 to 0.82785, saving model to best_model.h5


182/182 - 103s - 564ms/step - accuracy: 0.9334 - loss: 0.1912 - val_accuracy: 0.8278 - val_loss: 0.4473
Epoch 5/100

Epoch 5: val_accuracy improved from 0.82785 to 0.84122, saving model to best_model.h5


182/182 - 89s - 491ms/step - accuracy: 0.9311 - loss: 0.1987 - val_accuracy: 0.8412 - val_loss: 0.4498
Epoch 6/100

Epoch 6: val_accuracy improved from 0.84122 to 0.84225, saving model to best_model.h5


182/182 - 19s - 105ms/step - accuracy: 0.9495 - loss: 0.1569 - val_accuracy: 0.8422 - val_loss: 0.4691
Epoch 7/100

Epoch 7: val_accuracy did not improve from 0.84225
182/182 - 147s - 810ms/step - accuracy: 0.9523 - loss: 0.1457 - val_accuracy: 0.8313 - val_loss: 0.4816
Epoch 8/100

Epoch 8: val_accuracy did not improve from 0.84225
182/182 - 91s - 500ms/step - accuracy: 0.9499 - loss: 0.1523 - val_accuracy: 0.8422 - val_loss: 0.4762
Epoch 9/100

Epoch 9: val_accuracy improved from 0.84225 to 0.84396, saving model to best_model.h5


182/182 - 14s - 75ms/step - accuracy: 0.9335 - loss: 0.1853 - val_accuracy: 0.8440 - val_loss: 0.4857
Epoch 10/100

Epoch 10: val_accuracy did not improve from 0.84396
182/182 - 95s - 523ms/step - accuracy: 0.9639 - loss: 0.1194 - val_accuracy: 0.8179 - val_loss: 0.5494
Epoch 11/100

Epoch 11: val_accuracy did not improve from 0.84396
182/182 - 90s - 496ms/step - accuracy: 0.9584 - loss: 0.1267 - val_accuracy: 0.8302 - val_loss: 0.5346
Epoch 12/100

Epoch 12: val_accuracy did not improve from 0.84396
182/182 - 14s - 75ms/step - accuracy: 0.9478 - loss: 0.1324 - val_accuracy: 0.8371 - val_loss: 0.5227
Epoch 13/100

Epoch 13: val_accuracy did not improve from 0.84396
182/182 - 92s - 504ms/step - accuracy: 0.9696 - loss: 0.1012 - val_accuracy: 0.8416 - val_loss: 0.5490
Epoch 14/100

Epoch 14: val_accuracy did not improve from 0.84396
182/182 - 90s - 495ms/step - accuracy: 0.9671 - loss: 0.1117 - val_accuracy: 0.8313 - val_loss: 0.5180
Epoch 14: early stopping
Restoring model weights from 

Evaluate

In [ ]:
y_test_probs = model.predict(pad_test)
y_pred = np.argmax(y_test_probs, axis=1)
y_true = np.argmax(y_test_encode, axis=1)

target_names = le.inverse_transform([0,1,2])

print(classification_report(y_true, y_pred, target_names=target_names))